In [1]:
# Imports e instalações

import pandas as pd
import numpy as np

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

#!pip install scikit-learn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Métricas
from sklearn.metrics import (accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report, f1_score)


In [2]:
df = pd.read_csv(r'C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\australia_clima_v4.csv', sep=',')
df = df.sort_values("data").reset_index(drop=True)
df

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,ano,mes,dia,trimestre_quarter,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna
0,2007-11-01,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,...,2007,11,1,4,False,False,False,False,False,False
1,2007-11-02,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,2007,11,2,4,False,False,False,False,False,False
2,2007-11-03,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,...,2007,11,3,4,False,False,False,False,False,False
3,2007-11-04,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,...,2007,11,4,4,False,False,False,False,False,False
4,2007-11-05,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,...,2007,11,5,4,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56565,2017-06-25,Perth,6.3,17.0,0.0,1.6,7.9,E,26.0,SE,...,2017,6,25,2,False,False,False,False,False,False
56566,2017-06-25,Watsonia,7.9,13.0,0.0,2.8,3.8,NNW,39.0,N,...,2017,6,25,2,False,False,False,False,False,False
56567,2017-06-25,Sydney,7.6,19.3,0.0,3.4,9.4,W,35.0,W,...,2017,6,25,2,False,False,False,False,False,False
56568,2017-06-25,PerthAirport,4.9,16.8,0.0,1.6,7.9,E,39.0,E,...,2017,6,25,2,False,False,False,False,False,False


In [3]:
# Estatísticas Gerais
df.info()
df.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56570 entries, 0 to 56569
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   data                     56570 non-null  object 
 1   localidade               56570 non-null  object 
 2   temp_min                 56570 non-null  float64
 3   temp_max                 56570 non-null  float64
 4   chuva_mm                 56570 non-null  float64
 5   evaporacao               56570 non-null  float64
 6   horas_sol                56570 non-null  float64
 7   vento_rajada_direcao     56570 non-null  object 
 8   vento_rajada_velocidade  56570 non-null  float64
 9   vento_direcao_9h         56570 non-null  object 
 10  vento_direcao_15h        56570 non-null  object 
 11  vento_veloc_9h           56570 non-null  float64
 12  vento_veloc_15h          56570 non-null  float64
 13  umidade_9h               56570 non-null  float64
 14  umidade_15h           

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,ano,mes,dia,trimestre_quarter,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna
count,56570,56570,56570.000000,56570.000000,56570.000000,56570.000000,56570.000000,56570,56570.000000,56570,...,56570.000000,56570.000000,56570.000000,56570.000000,56570,56570,56570,56570,56570,56570
unique,3416,26,NaN,NaN,NaN,NaN,NaN,16,NaN,16,...,NaN,NaN,NaN,NaN,1,1,1,1,1,1
top,2010-06-09,Darwin,NaN,NaN,NaN,NaN,NaN,E,NaN,N,...,NaN,NaN,NaN,NaN,False,False,False,False,False,False
freq,25,3063,NaN,NaN,NaN,NaN,NaN,4531,NaN,4981,...,NaN,NaN,NaN,NaN,56570,56570,56570,56570,56570,56570
mean,NaN,NaN,13.458626,24.210032,2.131868,5.499295,7.732842,NaN,40.866343,NaN,...,2012.219445,6.423369,15.721584,2.477903,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,6.417872,6.973173,7.014007,3.696565,3.758660,NaN,13.340615,NaN,...,2.450910,3.450963,8.783534,1.121646,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,-6.700000,4.100000,0.000000,0.000000,0.000000,NaN,9.000000,NaN,...,2007.000000,1.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,8.600000,18.700000,0.000000,2.800000,5.000000,NaN,31.000000,NaN,...,2010.000000,3.000000,8.000000,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,13.200000,23.900000,0.000000,5.000000,8.600000,NaN,39.000000,NaN,...,2012.000000,6.000000,16.000000,2.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,18.400000,29.700000,0.600000,7.400000,10.700000,NaN,48.000000,NaN,...,2014.000000,9.000000,23.000000,3.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Tamanho do dataset
df.shape

(56570, 33)

In [5]:
# Valores Nulos
df.isnull().sum()

data                       0
localidade                 0
temp_min                   0
temp_max                   0
chuva_mm                   0
evaporacao                 0
horas_sol                  0
vento_rajada_direcao       0
vento_rajada_velocidade    0
vento_direcao_9h           0
vento_direcao_15h          0
vento_veloc_9h             0
vento_veloc_15h            0
umidade_9h                 0
umidade_15h                0
pressao_9h                 0
pressao_15h                0
nuvem_9h                   0
nuvem_15h                  0
temp_9h                    0
temp_15h                   0
choveu_hoje_fex            0
chove_amanha_vtr           0
ano                        0
mes                        0
dia                        0
trimestre_quarter          0
horas_sol_isna             0
evaporacao_isna            0
nuvem_9h_isna              0
nuvem_15h_isna             0
pressao_9h_isna            0
pressao_15h_isna           0
dtype: int64

In [6]:
df['chove_amanha_vtr'].value_counts(normalize=True)

chove_amanha_vtr
No     0.779477
Yes    0.220523
Name: proportion, dtype: float64

In [7]:
# Normalização da variável alvo
df['chove_amanha_vtr'] = df['chove_amanha_vtr'].map({'Yes': 1, 'No': 0})
df['chove_amanha_vtr']

0        1
1        1
2        1
3        1
4        0
        ..
56565    0
56566    0
56567    0
56568    0
56569    0
Name: chove_amanha_vtr, Length: 56570, dtype: int64

In [8]:
df['chove_amanha_vtr'].value_counts(normalize=True)

chove_amanha_vtr
0    0.779477
1    0.220523
Name: proportion, dtype: float64

In [9]:
df.columns.tolist()


['data',
 'localidade',
 'temp_min',
 'temp_max',
 'chuva_mm',
 'evaporacao',
 'horas_sol',
 'vento_rajada_direcao',
 'vento_rajada_velocidade',
 'vento_direcao_9h',
 'vento_direcao_15h',
 'vento_veloc_9h',
 'vento_veloc_15h',
 'umidade_9h',
 'umidade_15h',
 'pressao_9h',
 'pressao_15h',
 'nuvem_9h',
 'nuvem_15h',
 'temp_9h',
 'temp_15h',
 'choveu_hoje_fex',
 'chove_amanha_vtr',
 'ano',
 'mes',
 'dia',
 'trimestre_quarter',
 'horas_sol_isna',
 'evaporacao_isna',
 'nuvem_9h_isna',
 'nuvem_15h_isna',
 'pressao_9h_isna',
 'pressao_15h_isna']

In [10]:
# Convertendo variável 'choveu_hoje_fex' em numérica
df['choveu_hoje'] = df['choveu_hoje_fex'].map({'Yes': 1, 'No': 0})

# Calculando a amplitude térmica
df['amplitude_termica'] = df['temp_max'] - df['temp_min']

# Calculando a umidade média
df['umidade_media'] = (df['umidade_9h'] + df['umidade_15h']) / 2

# Variação da pressão atmosférica
df['variacao_pressao'] = df['pressao_9h'] - df['pressao_15h']



In [11]:
# Conferindo valores nulos
df.isna().sum()

data                       0
localidade                 0
temp_min                   0
temp_max                   0
chuva_mm                   0
evaporacao                 0
horas_sol                  0
vento_rajada_direcao       0
vento_rajada_velocidade    0
vento_direcao_9h           0
vento_direcao_15h          0
vento_veloc_9h             0
vento_veloc_15h            0
umidade_9h                 0
umidade_15h                0
pressao_9h                 0
pressao_15h                0
nuvem_9h                   0
nuvem_15h                  0
temp_9h                    0
temp_15h                   0
choveu_hoje_fex            0
chove_amanha_vtr           0
ano                        0
mes                        0
dia                        0
trimestre_quarter          0
horas_sol_isna             0
evaporacao_isna            0
nuvem_9h_isna              0
nuvem_15h_isna             0
pressao_9h_isna            0
pressao_15h_isna           0
choveu_hoje                0
amplitude_term

In [12]:
df[['choveu_hoje', 'amplitude_termica', 'umidade_media', 'variacao_pressao']].describe()

,choveu_hoje,amplitude_termica,umidade_media,variacao_pressao
count,56570.000000,56570.000000,56570.000000,56570.000000
mean,0.220983,10.751405,57.753597,2.443039
std,0.414913,4.766209,17.767473,1.912216
min,0.000000,-3.200000,0.000000,-10.800000
25%,0.000000,7.100000,47.000000,1.400000
50%,0.000000,10.100000,59.000000,2.700000
75%,0.000000,14.000000,70.000000,3.700000
max,1.000000,31.400000,100.000000,15.100000


In [13]:
df

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna,choveu_hoje,amplitude_termica,umidade_media,variacao_pressao
0,2007-11-01,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,...,False,False,False,False,False,False,0,16.3,48.5,4.7
1,2007-11-02,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,False,False,False,False,False,False,1,12.9,58.0,4.0
2,2007-11-03,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,...,False,False,False,False,False,False,1,9.7,75.5,2.3
3,2007-11-04,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,...,False,False,False,False,False,False,1,2.2,59.0,-1.5
4,2007-11-05,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,...,False,False,False,False,False,False,1,8.5,58.5,-0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56565,2017-06-25,Perth,6.3,17.0,0.0,1.6,7.9,E,26.0,SE,...,False,False,False,False,False,False,0,10.7,62.0,2.6
56566,2017-06-25,Watsonia,7.9,13.0,0.0,2.8,3.8,NNW,39.0,N,...,False,False,False,False,False,False,0,5.1,68.5,2.3
56567,2017-06-25,Sydney,7.6,19.3,0.0,3.4,9.4,W,35.0,W,...,False,False,False,False,False,False,0,11.7,52.5,3.2
56568,2017-06-25,PerthAirport,4.9,16.8,0.0,1.6,7.9,E,39.0,E,...,False,False,False,False,False,False,0,11.9,56.5,2.5


In [14]:
# Separando X e y
alvo = 'chove_amanha_vtr'

X = df.drop(columns=[alvo])
y = df[alvo]



In [15]:
colunas_categoricas = [
    'localidade',
    'vento_rajada_direcao',
    'vento_direcao_9h',
    'vento_direcao_15h'
]

colunas_numericas = [
    'temp_min', 'temp_max', 'chuva_mm', 'evaporacao', 'horas_sol',
    'vento_rajada_velocidade', 'vento_veloc_9h', 'vento_veloc_15h',
    'umidade_9h', 'umidade_15h',
    'pressao_9h', 'pressao_15h',
    'nuvem_9h', 'nuvem_15h',
    'temp_9h', 'temp_15h',
    'ano', 'mes', 'dia', 'trimestre_quarter',
    # flags isna
    #'horas_sol_isna', 'evaporacao_isna',
    #'nuvem_9h_isna', 'nuvem_15h_isna',
    #'pressao_9h_isna', 'pressao_15h_isna',
    # cols/features novas
    'choveu_hoje', 'amplitude_termica', 'umidade_media', 'variacao_pressao'
]


In [16]:
colunas_categoricas

['localidade', 'vento_rajada_direcao', 'vento_direcao_9h', 'vento_direcao_15h']

In [17]:
colunas_numericas

['temp_min',
 'temp_max',
 'chuva_mm',
 'evaporacao',
 'horas_sol',
 'vento_rajada_velocidade',
 'vento_veloc_9h',
 'vento_veloc_15h',
 'umidade_9h',
 'umidade_15h',
 'pressao_9h',
 'pressao_15h',
 'nuvem_9h',
 'nuvem_15h',
 'temp_9h',
 'temp_15h',
 'ano',
 'mes',
 'dia',
 'trimestre_quarter',
 'choveu_hoje',
 'amplitude_termica',
 'umidade_media',
 'variacao_pressao']

In [18]:
colunas_categoricas = [c for c in colunas_categoricas if c in X.columns]
colunas_numericas = [c for c in colunas_numericas if c in X.columns]

In [19]:
colunas_categoricas

['localidade', 'vento_rajada_direcao', 'vento_direcao_9h', 'vento_direcao_15h']

In [20]:
colunas_numericas

['temp_min',
 'temp_max',
 'chuva_mm',
 'evaporacao',
 'horas_sol',
 'vento_rajada_velocidade',
 'vento_veloc_9h',
 'vento_veloc_15h',
 'umidade_9h',
 'umidade_15h',
 'pressao_9h',
 'pressao_15h',
 'nuvem_9h',
 'nuvem_15h',
 'temp_9h',
 'temp_15h',
 'ano',
 'mes',
 'dia',
 'trimestre_quarter',
 'choveu_hoje',
 'amplitude_termica',
 'umidade_media',
 'variacao_pressao']

In [21]:
# Pré-processamento e pipeline
preprocessamento_numerico = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])

preprocessamento_categorico = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessador = ColumnTransformer(transformers=[
        ('num', preprocessamento_numerico, colunas_numericas),
        ('cat', preprocessamento_categorico, colunas_categoricas)
    ], remainder='drop'
)

In [22]:
# Split dos dados
ponto_corte = int(len(df) * 0.7)

X_treino = X.iloc[:ponto_corte]
y_treino = y.iloc[:ponto_corte]

X_teste  = X.iloc[ponto_corte:]
y_teste  = y.iloc[ponto_corte:]

# Fazendo último check no NaN
mask_treino = y_treino.notna()
mask_teste  = y_teste.notna()

X_treino, y_treino = X_treino[mask_treino], y_treino[mask_treino]
X_teste,  y_teste  = X_teste[mask_teste],  y_teste[mask_teste]

In [ ]:
# Primeiro, eu defino onde vou cortar o dataset no tempo
# Aqui, uso 70% dos dados mais antigos para treino e 30% dos dados mais recentes para teste

split_point = int(len(X) * 0.7)

# Aqui eu separo as variáveis preditoras de treino
# Estou pegando tudo do início até o ponto de corte
X_train = X.iloc[:split_point]

# Aqui ficam as variáveis preditoras de teste, somente com dados "do futuro" em relação ao treino
X_test = X.iloc[split_point:]


# Agora faço a mesma separação, mas para o alvo (y)

# Alvo de treino
y_train = y.iloc[:split_point]

# Alvo de teste
y_test = y.iloc[split_point:]

# Neste passo, eu separo as colunas numéricas das categóricas, porque cada tipo de dado precisa de um tratamento diferente

# Variáveis numérica >>>
# São colunas com valores numéricos como temperatura, pressão, umidade, etc
variaveis_numericas = X.select_dtypes(include=['int64', 'float64']).columns

# Variáveis categóricas >>>
# São colunas com texto ou categorias,
# como localidade ou direção do vento.
variaveis_categoricas = X.select_dtypes(include=['object', 'category']).columns



# Aqui eu defino como vou tratar as variáveis numéricas
# antes de passá-las para o modelo.

# 001 ///// preencher valores nulos com a mediana.
# Eu escolhi a mediana porque ela é mais robusta a outliers

# 002 padronizar os dados (StandardScaler) deixando média = 0 e desvio padrão = 1
# Isso é importante principalmente para modelos lineares

pipeline_numerico = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')), ('scaler', StandardScaler())])


# transformo cada categoria em colunas binárias
# Uso handle_unknown='ignore' para evitar erro caso apareça uma categoria nova no conjunto de teste
pipeline_categorico = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore'))])

# O ColumnTransformer me permite aplicar o pipeline numérico apenas nas colunas numéricas e o pipeline categórico apenas nas colunas categóricas
preprocessador = ColumnTransformer(transformers=[('num', pipeline_numerico, variaveis_numericas), ('cat', pipeline_categorico, variaveis_categoricas)])

# Aqui eu começo com um modelo de Regressão Logística, porque ele é simples, interpretável e ótimo para servir como baseline
# Uso class_weight='balanced' porque a classe "chove amanhã" é minoritária no dataset

modelo = LogisticRegression(
    max_iter=1000,
    class_weight='balanced',
    random_state=42
)



# Neste pipeline final, eu junto todo o pré-processamento e o modelo de classificação
# Isso garante que o pré-processamento seja aprendido apenas com os dados de treino, evitando vazamento de informação
pipeline_modelo = Pipeline(steps=[('preprocessador', preprocessador),('modelo', modelo)])



# Aqui eu treino o pipeline completo.
# Internamente, ele aprende:
# - imputação
# - escalonamento
# - codificação categórica
# - e, por fim, ajusta o modelo

pipeline_modelo.fit(X_train, y_train)


# Depois do treino, eu gero as previsões
# para o conjunto de teste.
y_pred = pipeline_modelo.predict(X_test)

# Agora avalio o desempenho do modelo usando as métricas mais importantes para um problema de classificação binária.
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))

# A matriz de confusão me ajuda a entender onde o modelo está errando e acertando.
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# O classification report traz um resumo completo
# das métricas por classe.
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.8175122267397325
Precision: 0.557363370880267
Recall: 0.7326569783383603
F1: 0.6331003435611894

Confusion Matrix:
[[11202  2122]
 [  975  2672]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.84      0.88     13324
           1       0.56      0.73      0.63      3647

    accuracy                           0.82     16971
   macro avg       0.74      0.79      0.76     16971
weighted avg       0.84      0.82      0.83     16971



In [ ]:
# Como esse é um problema temporal, eu mantenho exatamente
# o mesmo split usado antes (70% treino / 30% teste),
# respeitando a ordem dos dados no tempo.

split_point = int(len(X) * 0.7)

X_train = X.iloc[:split_point]
X_test  = X.iloc[split_point:]

y_train = y.iloc[:split_point]
y_test  = y.iloc[split_point:]

# Aqui eu separo as variáveis numéricas das categóricas, porque cada uma exige um tratamento diferente no pipeline
variaveis_numericas = X.select_dtypes(include=['int64', 'float64']).columns
variaveis_categoricas = X.select_dtypes(include=['object', 'category']).columns
pipeline_numerico = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])


# Para dados categóricos
# Preencho valores nulos com o valor mais frequente
# Aplico One-Hot Encoding
# Uso handle_unknown='ignore' para evitar erro se aparecer categoria nova no conjunto de teste.
pipeline_categorico = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore'))])



# O ColumnTransformer me permite aplicar cada pipeline apenas no tipo correto de variável.
preprocessador = ColumnTransformer(
    transformers=[('num', pipeline_numerico, variaveis_numericas), ('cat', pipeline_categorico, variaveis_categoricas)])


# Random Forest
modelo = RandomForestClassifier(
    n_estimators=400,          # número de árvores
    random_state=42,
    n_jobs=-1,                 # usa todos os núcleos
    class_weight='balanced'    # ajuda com classe minoritária (chuva)
)

# Aqui eu uno todo o pré-processamento com o modelo.
# Isso garante que nenhuma informação do teste vaze para o treino.

pipeline_modelo = Pipeline(steps=[('preprocessador', preprocessador), ('modelo', modelo)])


# Agora eu treino o Random Forest.
# Cada árvore é treinada com subconjuntos diferentes dos dados e das features, e o resultado final é um voto majoritário entre todas elas.
pipeline_modelo.fit(X_train, y_train)


# Depois do treino, eu gero as previsões no conjunto de teste.
y_pred = pipeline_modelo.predict(X_test)

# Aqui eu avalio o modelo usando as mesmas métricas da regressão logística, para poder comparar diretamente.

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))

# A matriz de confusão me mostra exatamente onde o modelo acerta e onde erra.
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Relatório completo por classe
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.8495669082552589
Precision: 0.7793667007150153
Recall: 0.418426103646833
F1: 0.544513826940232

Confusion Matrix:
[[12892   432]
 [ 2121  1526]]

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.97      0.91     13324
           1       0.78      0.42      0.54      3647

    accuracy                           0.85     16971
   macro avg       0.82      0.69      0.73     16971
weighted avg       0.84      0.85      0.83     16971



In [ ]:
# Como o problema é temporal (prever se chove amanhã), eu continuo respeitando a ordem dos dados.
# Uso exatamente o mesmo split anterior (70% treino / 30% teste)
# para garantir comparabilidade entre modelos.
split_point = int(len(X) * 0.7)

X_train = X.iloc[:split_point]
X_test  = X.iloc[split_point:]

y_train = y.iloc[:split_point]
y_test  = y.iloc[split_point:]


# Eu separo variáveis numéricas e categóricas,
# porque cada tipo exige um tratamento específico
# antes de entrar no modelo.
variaveis_numericas = X.select_dtypes(include=['int64', 'float64']).columns
variaveis_categoricas = X.select_dtypes(include=['object', 'category']).columns


# Para as variáveis numéricas, eu:
# - preencho valores nulos com a mediana
# Gradient Boosting NÃO precisa de padronização,
# então não uso StandardScaler aqui.
pipeline_numerico = Pipeline(steps=[('imputer', SimpleImputer(strategy='median'))])                          


# Para variáveis categóricas, eu:
# - preencho valores nulos com o valor mais frequente
# - aplico One-Hot Encoding
# Uso handle_unknown='ignore' para evitar erro
# caso apareça uma categoria nova no conjunto de teste.
pipeline_categorico = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('encoder', OneHotEncoder(handle_unknown='ignore'))])

# O ColumnTransformer me permite aplicar
# cada pipeline apenas ao tipo correto de variável,
# mantendo tudo organizado e sem vazamento de dados.
preprocessador = ColumnTransformer(
    transformers=[
        ('num', pipeline_numerico, variaveis_numericas),
        ('cat', pipeline_categorico, variaveis_categoricas)
    ]
)


# Aqui eu escolho Gradient Boosting porque:

modelo = GradientBoostingClassifier(
    n_estimators=300,      # número de árvores (boosting rounds)
    learning_rate=0.03,    # passo pequeno para evitar overfitting
    max_depth=3,           # árvores rasas (controle de complexidade)
    random_state=42
)


# Aqui eu uno todo o pré-processamento com o modelo.
# Isso garante que tudo seja aprendido apenas no treino.
pipeline_modelo = Pipeline(
    steps=[
        ('preprocessador', preprocessador),
        ('modelo', modelo)
    ]
)


# Agora eu treino o Gradient Boosting.
# Cada árvore tenta corrigir os erros residuais deixados pelas árvores anteriores.
pipeline_modelo.fit(X_train, y_train)


# Depois do treino, eu faço as previsões no conjunto de teste.
y_pred = pipeline_modelo.predict(X_test)

# Avalio o modelo usando as mesmas métricas para poder comparar diretamente com logística e RF.
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))

# A matriz de confusão mostra claramente o trade-off entre falsos positivos e falsos negativos.
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Relatório detalhado por classe
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.8531023510694714
Precision: 0.7418273260687342
Recall: 0.48533040855497667
F1: 0.5867727498756837

Confusion Matrix:
[[12708   616]
 [ 1877  1770]]

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.95      0.91     13324
           1       0.74      0.49      0.59      3647

    accuracy                           0.85     16971
   macro avg       0.81      0.72      0.75     16971
weighted avg       0.84      0.85      0.84     16971

